In [5]:
!pip install dash
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from dash import Dash, html, dcc, Input, Output
import plotly.graph_objs as go

In [6]:
# --- Generate a larger dataset (50 points) ---
np.random.seed(42)  # reproducibility
marketing_spend = np.linspace(50, 500, 50)
profit = 5 * marketing_spend + 200 + np.random.normal(0, 200, 50)  # linear-ish with noise

dataset = pd.DataFrame({
    'Marketing Spend': marketing_spend,
    'Profit': profit
})
x = dataset[['Marketing Spend']].values
y = dataset['Profit'].values

# --- Define model ---
def define_model(degree):
    poly_reg = PolynomialFeatures(degree)
    x_poly = poly_reg.fit_transform(x)
    lin_reg = LinearRegression()
    lin_reg.fit(x_poly, y)
    return lin_reg, poly_reg

def predict_value(lin_reg, poly_reg, spend):
    spend_poly = poly_reg.transform([[spend]])
    return lin_reg.predict(spend_poly)[0]

# --- Dash App ---
app = Dash(__name__)
app.layout = html.Div([
    html.H1("Actual vs Predicted Profits in One Graph"),

    html.Div([
        html.Label("Enter the Marketing Spend:"),
        dcc.Input(id='spend_input', type='number', value=100, step=10),
    ], style={'marginBottom':20}),

    html.Div([
        html.Label("Select a Polynomial Degree:"),
        dcc.Dropdown(id='dropdown',
            options=[{'label': str(d), 'value': d} for d in range(1,7)],
            value=4,
            clearable=False),
    ], style={'marginBottom':20, 'width':'40%'}),

    dcc.Graph(id='poly_graph'),
    html.Div(id='prediction_text', style={'marginTop':20, 'fontSize':20, 'color':'blue'})
])

# --- Callback ---
@app.callback(
    [Output('poly_graph', 'figure'),
     Output('prediction_text', 'children')],
    [Input('spend_input', 'value'),
     Input('dropdown', 'value')]
)
def update_graph(spend_val, degree):
    if spend_val is None:
        spend_val = 100

    lin_reg, poly_reg = define_model(degree)
    predicted_profit = predict_value(lin_reg, poly_reg, spend_val)

    # Range for smooth polynomial line
    x_range = np.linspace(x.min(), x.max(), 200).reshape(-1, 1)
    y_pred = lin_reg.predict(poly_reg.transform(x_range))

    # --- Single Graph: Scatter + Line ---
    fig = go.Figure()

    # Actual data (scatter)
    fig.add_trace(go.Scatter(
        x=dataset['Marketing Spend'], y=dataset['Profit'],
        mode='markers',
        marker=dict(color='red', size=10, line=dict(width=1, color='black')),
        name="Actual Profit"
    ))

    # Polynomial regression line
    fig.add_trace(go.Scatter(
        x=x_range.ravel(), y=y_pred,
        mode='lines',
        line=dict(color='blue', width=2),
        name=f"Polynomial Fit (deg={degree})"
    ))

    # Prediction point
    fig.add_trace(go.Scatter(
        x=[spend_val], y=[predicted_profit],
        mode='markers+text',
        text=[f"{predicted_profit:.2f}k"],
        textposition="top center",
        marker=dict(color='green', size=12, symbol="diamond"),
        name="Prediction"
    ))

    # Layout
    fig.update_layout(
        title="Actual vs Predicted Profit",
        xaxis_title="Marketing Spend ($k)",
        yaxis_title="Profit ($k)",
        width=900, height=500,
        legend=dict(x=0.05, y=0.95)
    )

    return fig, f"Predicted profit for spend = {spend_val}k with degree {degree} : {predicted_profit:.2f}k"

# --- Run app ---
if __name__ == '__main__':
    app.run(debug=True)



<IPython.core.display.Javascript object>